In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
%matplotlib inline
pd.options.display.max_columns = 15


In [ ]:
filepath = "C:\\Users\\seanp\\\WomenFootballCMEffectiveness\\open-data\\data\\matches\\"
competitions = ["37"]
league_match_list = []
for comp in competitions:
    path = filepath+comp
    print(path)
    league_match_list.extend(glob.glob(path+"\\*.json"))
league_match_list = ["C:\\Users\\seanp\\WomenFootballCMEffectiveness\\open-data\\data\\matches\\37\\4.json"]
#https://www.statsperform.com/resource/how-impactful-are-line-breaking-passes/
#very important math stuff for futbol analytics

In [ ]:
#extracting match specific information
eventspath = "C:\\Users\\seanp\\WomenFootballCMEffectiveness\\open-data\\data\\events\\"
lineup_path = "C:\\Users\\seanp\\WomenFootballCMEffectiveness\\open-data\\data\\lineups\\"
match_list = []
lineup_list = []
temp_match = ""
for season in league_match_list:
    df_matches=pd.read_json(season)
    for i in range(0,len(df_matches)):  
        temp_match = str(df_matches.iloc[i]['match_id'])
        match_path = eventspath+temp_match+".json"
        individual_lineup_path = lineup_path+temp_match+".json"
        match_list.extend(glob.glob(match_path))
        lineup_list.extend(glob.glob(individual_lineup_path))
lineup_list


In [ ]:
#https://towardsdatascience.com/all-pandas-json-normalize-you-should-know-for-flattening-json-13eae1dfb7dd
center_mid_positions = [9,10,11,13,14,15,18,19,20]
column_names = ['player_id','player_name','position_id','position_name','team_name','team_id']
players = pd.DataFrame(columns = column_names)
counter = 0
for lineup in lineup_list:
    f = open(lineup)
    data = json.load(f)
    player_info = pd.json_normalize(data,record_path = ['lineup'], meta = ['team_name','team_id'])
    player_info['position_id'] = 0
    player_info['position_name'] = ''
    for i in range(0,len(player_info)):
        try:
            x = player_info.loc[i,'positions'][0]['position_id']
            y = player_info.loc[i,'positions'][0]['position']
            if(x in center_mid_positions):
                player_info.loc[i,'position_id'] = x
                player_info.loc[i,'position_name'] = y
        except:
            print(counter)
            counter += 1
            print(player_info.loc[i,'player_name'] + " bad data in match: " + lineup)
    player_info = player_info[column_names]
    players = pd.merge(players,player_info,how = 'outer',validate='one_to_one')
    players = players.drop_duplicates(subset='player_id')
        #player_info.iloc[i]['postion_name'] = player_info.iloc[i]['positions'][1]['position']
#had to junk teams because of players getting transferred in season
players = players[~(players['position_id']==0)]
players

In [ ]:
#Determining fuck-ups
#df_goals_centermids = df_goals[(df_errors['position.id'].isin(center_mid_positions) & df_errors)]
def cm_Errors():
    for i in range(0,len(df_errors)):
        temp_id_error = df_errors.iloc[i]
        try:
            #temp_id_player = df_goals_centermids.loc[df_goals_centermids['id'] == temp_id_goal]
            players.loc[players['player_id'] == (temp_id_error['player.id'].astype('int')), 'Errors'] += 1
        except:
            pass

In [ ]:
#Code for how many goals center mids score
def cm_goals_function():
    for i in range(0,len(df_goals)):
        temp_id_goal = df_goals.iloc[i]
        try:
            #temp_id_player = df_goals_centermids.loc[df_goals_centermids['id'] == temp_id_goal]
            players.loc[players['player_id'] == (temp_id_goal['player.id'].astype('int')), 'Goals'] += 1
        except:
            pass


In [ ]:
#Code for key-passes made by center-mid
def key_Passes():
    for i in range(0,len(df_shots)):
        temp_id_pass = df_shots.iloc[i]['shot.key_pass_id']
        #print(type(temp_id))
        #print(type(temp_id))
        #print(temp_id)
        try:    
            temp_id_player = df_passes_centermids.loc[df_passes_centermids['id'] == temp_id_pass]
            temp_id_player = temp_id_player['player.id'].astype(int)
            players.loc[players['player_id'] == temp_id_player.iloc[0], 'Key_Passes_Made'] += 1
        except:
            pass
 

In [ ]:



def generate_rating_helper(Pass_Percentage, Goals, Key_Passes_Made, Errors, Forward_Passes_Zone_5_8, Incomplete_Backwards_Pass):
    good_passes = (Forward_Passes_Zone_5_8-Key_Passes_Made)*.5 + Key_Passes_Made
    # max function protects negative values the lowest someone can get for the first composite value is a 1 which is already pretty terrible
    return ((max(Goals-Errors-Incomplete_Backwards_Pass,1) + good_passes/2) * Pass_Percentage)
def generate_rating():
    players['Rating'] = 0
    players['Rating'] = players.apply(lambda x: generate_rating_helper(x['Pass_Percentage'], x['Goals'], x['Key_Passes_Made'], x['Errors'], x['Forward_Passes_Zone_5_8'], x['Incomplete_Backwards_Pass']), axis=1)
    #players['Rating'] = players['Pass_Percentage'] * players['Goals']* 2 * players['Key_Passes_Made']
 

In [ ]:
# Psuedo line breaking - link to another project that actually does this https://www.statsperform.com/resource/how-impactful-are-line-breaking-passes/
# This is a poor man's version that I think surprisely does a good job at measuring because
# of the angle that says the pass must go forward
# Defining zone 5/8 the defensive third in between lines zone
def forward_angle(angle):
    if((-1.56 < angle) & (angle < 1.56 )):
        return True
    else:
        return False
def zone_5_8(player):
    if((player['team.name'] == left_side_team) & (player['period'] == 1)):
        if((24 < player['location'][0] < 49) & (17 < player['location'][1])):
            #print('I am in the zone')
            return True
    if((player['team.name'] == left_side_team) & (player['period'] == 2)):
        if((69 < player['location'][0] < 94) & (17 < player['location'][1])):
            #print('I am in the zone')
            return True
    if((player['team.name'] == right_side_team) & (player['period'] == 1)):
        if((69 < player['location'][0] < 94) & (17 < player['location'][1])):
            #print('I am in the zone')
            return True
    if((player['team.name'] == right_side_team) & (player['period'] == 1)):
        if((24 < player['location'][0] < 49) & (17 < player['location'][1])):
            #print('I am in the zone')
            return True
    return False
def inzone_5_8(player_position): 
    if(zone_5_8(player_position)):
        return True
    else:
        return False
def passPercentage_and_Forward():
    for i in range(0,len(df_passes_centermids)):
        temp_id = df_passes_centermids.iloc[i]['player.id']
        try: 
            one_pass_result = df_passes_centermids.iloc[i]['pass.outcome.name']
            if(one_pass_result == 'Incomplete'):
                angle = df_passes_centermids.iloc[i]['pass.angle']
                if((~forward_angle(angle)) & (~np.isnan(angle)) & (inzone_5_8(df_passes_centermids.iloc[i]))):
                    #print(angle)
                    #print(one_pass_result)
                    players.loc[players['player_id'] == temp_id, 'Incomplete_Backwards_Pass'] += 1
        except:
            pass
        #how many passes while in zone 5_8 where the pass length is greater than 10 yards and must be forward
        try:
            #filterinfDataframe = dfObj[(dfObj['Sale'] > 30) & (dfObj['Sale'] < 33) ]
            if((forward_angle(df_passes_centermids.iloc[i]['pass.angle'])) & (df_passes_centermids.iloc[i]['pass.length'] > 10)):
                if(inzone_5_8(df_passes_centermids.iloc[i])):
                    players.loc[players['player_id'] == temp_id, 'Forward_Passes_Zone_5_8'] += 1
        except:
            pass
    for i in range(0,len(df_passes_centermids)):
        temp_id = int(df_passes_centermids.iloc[i]['player.id'])
        try:
            complete = df_passes_centermids.iloc[i]['pass.outcome.name']
            if(pd.isnull(complete)):
            #print(incomplete)
            #need to add line checking for incomplete though it seems to work without 
                players.loc[players['player_id'] == temp_id, 'Passes_Completed'] += 1
        except: 
            pass
        players.loc[players['player_id'] == temp_id, 'Passes_Attempted'] += 1
        # if(40 > df_passes.iloc[i]['pass']['end_location'][0] > 18) & (df_singular_match.iloc[i]['type']['name'] == 'Pass') & (df_singular_match.iloc[i]['pass']['outcome'][1] == 'Incomplete'):
        #     print('nothing')
    players['Pass_Percentage'] = players['Passes_Completed']/players['Passes_Attempted']


In [ ]:
def opposite_Team(first_team):
        if(first_team == team1):
            return team2
        if(first_team == team2):
            return team1
f = open(match_list[0])
data = json.load(f)
df_data = pd.json_normalize(data)
#team1 gets ball first that what must use different column names to access team2
team1_info = df_data.iloc[0]
team2_info = df_data.iloc[1]
team1 = team1_info['possession_team.name']
team2 = team2_info['team.name']
players['Goals'] = 0
players['Errors'] = 0
players['Passes_Attempted'] = 0
players['Passes_Attempted'] = pd.to_numeric(players['Passes_Attempted'], downcast="float")
players['Passes_Completed'] = 0
players['Passes_Completed'] = pd.to_numeric(players['Passes_Completed'], downcast="float")
players['Pass_Percentage'] = 0
players['Pass_Percentage'] = pd.to_numeric(players['Pass_Percentage'], downcast="float")
players['Incomplete_Backwards_Pass'] = 0
players['Key_Passes_Made'] = 0
players['Forward_Passes_Zone_5_8'] = 0
for i in range(0,len(match_list)):
    f = open(match_list[i])
    data = json.load(f)
    df_data = pd.json_normalize(data)
    team1_info = df_data.iloc[0]
    team2_info = df_data.iloc[1]
    #team1 gets ball first that what must use different column names to access team2
    # counter = 0
    # while(team_sides==0):
    #         try:
    #             if(df_singular_match.iloc[counter]['type']['Shot']):
    #                 team_sides = df_singular_match.iloc[counter]['player']['name']
    #         except:
    #             pass
    #         counter += 1
    # Create new columns that will focus on areas of interest same data
    df_shots = df_data[df_data['type.name'] == 'Shot']
    df_goals = df_data[df_data['shot.outcome.name'] == 'Goal']
    df_errors = df_data[(df_data['type.name'] == 'Error') & (df_data['position.id'].isin(center_mid_positions))]
    df_goals_centermids = df_goals[df_goals['position.id'].isin(center_mid_positions)]
    df_passes = df_data[df_data['type.name'] == 'Pass']
    df_passes_centermids = df_passes[df_passes['position.id'].isin(center_mid_positions)]
    df_passes_centermids['id'] = df_passes_centermids['id'].astype('string')
    #print(df_passes_centermids.dtypes)
    left_side_team = ''
    right_side_teame = ''
    #find first goal kick to determine halves play-pattern id is 7 play_pattern.id == 7
    df_goal_kicks = df_data[df_data['play_pattern.id'] == 7]
    if(df_goal_kicks.iloc[0]['location'][0] < 60):
        left_side_team = df_goal_kicks.iloc[0]['possession_team.name']
        right_side_team = opposite_Team(left_side_team)
    else:
        right_side_team = df_goal_kicks.iloc[0]['possession_team.name']
        left_side_team = opposite_Team(right_side_team)
    #print(left_side_team)
    #print(right_side_team)
    #filling out dataframe
    cm_Errors()
    cm_goals_function()
    key_Passes()
    passPercentage_and_Forward()
    generate_rating()
# Output of players is correct but rating is still needed
players